# Signal Fitting
We now have a successful background model in the form of a second-order polynomial. We now wish to fit an analytical function to our signal. In this way we can then assess how some figures of merit change based on different cuts or cutting methods since we have a way to estimate both signal and background within the same region.